<a href="https://colab.research.google.com/github/farsim-hossain/python-projects/blob/main/IMDB_Movie_Review_Classification_with_TF_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB Movie Review With Tf Hub Transfer learning

In [2]:
import os 
import numpy as np 

import tensorflow as tf 
import tensorflow_hub as hub 
import tensorflow_datasets as tfds 

## Downloading IMBD Dataset 
Splitting the dataset into 60% and 40% to end up with 15000 examples for training, 10000 examples for validation and 25000 for testing.


In [3]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5S83YD/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5S83YD/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5S83YD/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Exploring the data 
From the downloaded set, we will now see some from the train set. 


In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

train_examples_batch 
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## Building the model 

The neural network is created by stacking layers. In this example, the input data consists of sentences. The labels to predict are either 0 or 1.

One way to represent the text is to convert sentences into embeddings vectors. Use a pre-trained text embedding as the first layer, which will have three advantages:

You don't have to worry about text preprocessing,
Benefit from transfer learning,
the embedding has a fixed size, so it's simpler to process.
For this example you use a pre-trained text embedding model from TensorFlow Hub called google/nnlm-en-dim50/2.

Let's first create a Keras layer that uses a TensorFlow Hub model to embed the sentences, and try it out on a couple of input examples. Note that no matter the length of the input text, the output shape of the embeddings is: (num_examples, embedding_dimension).

In [5]:
# embedding layer 

embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape = [], dtype = tf.string, trainable = True)

hub_layer(train_examples_batch[:3]) # first 3 layers 

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [6]:
## The model 

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


## My favorite part - adding the loss function and optimization 

It seems that binary_crossentropy is better for probabilities (classification). For regression problems we might use mse.

In [7]:
model.compile(optimizer='adam', loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])



## Training The Model 
Train the model for 10 epochs in mini-batches of 512 samples. 


In [8]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs = 10, validation_data = validation_data.batch(512), verbose =1)

Epoch 1/10
30/30 [==============================] - 20s 617ms/step - loss: 0.6595 - accuracy: 0.5283 - val_loss: 0.5932 - val_accuracy: 0.6297
Epoch 2/10
30/30 [==============================] - 18s 571ms/step - loss: 0.5563 - accuracy: 0.6706 - val_loss: 0.4957 - val_accuracy: 0.7497
Epoch 3/10
30/30 [==============================] - 18s 585ms/step - loss: 0.4342 - accuracy: 0.7991 - val_loss: 0.4008 - val_accuracy: 0.8235
Epoch 4/10
30/30 [==============================] - 18s 574ms/step - loss: 0.3161 - accuracy: 0.8762 - val_loss: 0.3451 - val_accuracy: 0.8497
Epoch 5/10
30/30 [==============================] - 18s 589ms/step - loss: 0.2278 - accuracy: 0.9202 - val_loss: 0.3195 - val_accuracy: 0.8629
Epoch 6/10
30/30 [==============================] - 19s 612ms/step - loss: 0.1714 - accuracy: 0.9447 - val_loss: 0.3087 - val_accuracy: 0.8661
Epoch 7/10
30/30 [==============================] - 18s 579ms/step - loss: 0.1275 - accuracy: 0.9633 - val_loss: 0.3090 - val_accuracy: 0.8713

## Model Evaluation 



In [9]:
results = model.evaluate(test_data.batch(512), verbose =2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name,value))

49/49 - 3s - loss: 0.3526 - accuracy: 0.8542
loss: 0.353
accuracy: 0.854
